# Data Types - Lab


## Relationship between Population and World Cup Performance

![Russia 2018 branded soccer ball and trophy](https://curriculum-content.s3.amazonaws.com/data-science/images/world_cup.jpg)

<span>Photo by <a href="https://unsplash.com/@fznsr_?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Fauzan Saari</a> on <a href="https://unsplash.com/s/photos/soccer-world-cup?utm_source=unsplash&amp;utm_medium=referral&amp;utm_content=creditCopyText">Unsplash</a></span>

## Introduction

You are an analyst for GolStats Soccer Technologies, a soccer (football) analytics agency. They are interested in an analysis of various recent FIFA world cups -- specifically looking at the performance of a country in World cups in relation to their population.

You have been tasked with looking specifically at the sample of World Cup games in 2018 and seeing whether this has any relationship with the corresponding 2018 populations of the participating nations.


Your goal will be to create a data structure containing the requisite information for your analysis. Specifically, you'll build a dictionary where each key is the name of a country, and each value is a nested dictionary containing information about the number of 2018 World Cup wins and the country's population in 2018.

The final result will look something like this:
```
{
  'Argentina': { 'wins': 1, 'population': 44494502 },
  ...
  'Uruguay':   { 'wins': 4, 'population': 3449299  }
}
```

## Objectives

You will be using what you have learned about Python data types, data cleaning tasks, and CSV/JSON file formats to bear on this endeavor. You will be following a similar process to what you have been doing in the practice labs:
- Read serialized JSON and CSV data from files into Python objects
- Extract information from nested data structures
- Clean data (filtering, normalizing locations, converting types)
- Combine data from multiple sources into a single data structure


## Data Understanding

The data sources for this analysis will be pulled from two separate files located in this repository.

#### `world_cup_2018.json`

* **Source**: This dataset comes from [`football.db`](http://openfootball.github.io/), a "free and open public domain football database & schema for use in any (programming) language"
* **Contents**: Data about all games in the 2018 World Cup, including date, location (city and stadium), teams, goals scored (and by whom), and tournament group
* **Format**: Nested JSON data (dictionary containing a list of rounds, each of which contains a list of matches, each of which contains information about the teams involved and the points scored)

#### `country_populations.csv`

* **Source**: This dataset comes from a curated collection by [DataHub.io](https://datahub.io/core/population), originally sourced from the World Bank
* **Contents**: Data about populations by country for all available years from 1960 to 2018
* **Format**: CSV data, where each row contains a country name, a year, and a population

## Key Considerations

Before moving on to the next step, pause and think about the strategy for this analysis. Remember, our business question is:

- What is the relationship between the population of a country and their performance in the 2018 FIFA World Cup?

#### Unit of Analysis

First, what is our **unit of analysis**, and what is the **unique identifier**? In other words, what will one record in our final data structure represent, and what attribute uniquely describes it?

- *Our unit of analysis is a* ***country*** *and the unique identifier we'll use is the* ***country name***

#### Features

Next, what **features** are we analyzing? In other words, what attributes of each country are we interested in?

- *Our features are* ***2018 population*** *and* ***count of wins in the 2018 World Cup***

#### Dataset to Start With

Finally, which dataset should we **start** with? In this case, any record with missing data is not useful to us, so we want to start with the smaller dataset.

- There are only 32 countries that compete in the World Cup each year, compared to hundreds of countries in the world, so we should start with the* ***2018 World Cup*** *dataset. Then we can join it with the relevant records from the country population dataset.*

## Step 1: Loading the Data

Below we import the `json` and `csv` modules, which will be used for reading from `world_cup_2018.json` and `country_populations.csv`, respectively.

In [31]:
# CodeGrade step0
# Run this cell without changes
import json
import csv
import numpy as np

Using context managers and the appropriate modules:
- Load the data in the json file to a data structure world_cup_data.
- Load the data in the csv file to population_data. It should be a list of dictionaries.

In [32]:
# CodeGrade step1
# Replace None with your code
with open("world_cup_2018.json", encoding='utf8') as world_cup_file:
    world_cup_data = json.load(world_cup_file)

with open("country_populations.csv") as population_file:
    # make sure this is converted to a list
    population_data = list(csv.DictReader(population_file))

In [33]:

# Run this cell without changes

# Check that the overall data structure is a dictionary
assert type(world_cup_data) == dict

# Check that the dictionary has 2 keys, 'name' and 'rounds'
assert list(world_cup_data.keys()) == ['name', 'rounds']

In [34]:
# Run this cell without changes

# Check that the overall data structure is a list
assert type(population_data) == list

# Check that the 0th element is a dictionary
# (csv.DictReader interface differs slightly by Python version;
# element should be of type Dict)
assert type(population_data[0]) == dict

## Step 2. Exploring the Structure of the World Cup Data JSON

Let's start by exploring the structure of `world_cup_data`. Here is a pretty-printed preview of its contents:

```
{
  "name": "World Cup 2018",
  "rounds": [
    {
      "name": "Matchday 1",
      "matches": [
        {
          "num": 1,
          "date": "2018-06-14",
          "time": "18:00",
          "team1": { "name": "Russia",       "code": "RUS" },
          "team2": { "name": "Saudi Arabia", "code": "KSA" },
          "score1":  5,
          "score2":  0,
          "score1i": 2,
          "score2i": 0,
          "goals1": [
            { "name": "Gazinsky",   "minute": 12,              "score1": 1, "score2": 0 },
            { "name": "Cheryshev",  "minute": 43,              "score1": 2, "score2": 0 },
            { "name": "Dzyuba",     "minute": 71,              "score1": 3, "score2": 0 },
            { "name": "Cheryshev",  "minute": 90, "offset": 1, "score1": 4, "score2": 0 },
            { "name": "Golovin",    "minute": 90, "offset": 4, "score1": 5, "score2": 0 }
          ],
          "goals2": [],
          "group": "Group A",
          "stadium": { "key": "luzhniki", "name": "Luzhniki Stadium" },
          "city": "Moscow",
          "timezone": "UTC+3"
        }
      ]
    },
    {
      "name": "Matchday 2",
      "matches": [
        {
          "num": 2,
          "date": "2018-06-15",
          "time": "17:00",
          "team1": { "name": "Egypt",   "code": "EGY" },
          "team2": { "name": "Uruguay", "code": "URU" },
          "score1":  0,
          "score2":  1,
          "score1i": 0,
          "score2i": 0,
          "goals1": [],
          "goals2": [
            { "name": "Giménez",  "minute": 89,  "score1": 0, "score2": 1 }
          ],
          "group": "Group A",
          "stadium": { "key": "ekaterinburg", "name": "Ekaterinburg Arena" },          
          "city": "Ekaterinburg",
          "timezone": "UTC+5"
        },
        ...
      ],
    },
  ],  
}
```

As noted previously, `world_cup_data` is a dictionary with two keys, 'name' and 'rounds'.

Get the 2 highest level keys and save them as highest_lvl_keys below.

In [35]:
# CodeGrade step2
# Relace None with your code
highest_lvl_keys = world_cup_data.keys()

The value associated with the 'name' key is simply identifying the dataset.

In [36]:
# Run this cell without changes
print(world_cup_data['name'])
print(world_cup_data['rounds'][0])

World Cup 2018
{'name': 'Matchday 1', 'matches': [{'num': 1, 'date': '2018-06-14', 'time': '18:00', 'team1': {'name': 'Russia', 'code': 'RUS'}, 'team2': {'name': 'Saudi Arabia', 'code': 'KSA'}, 'score1': 5, 'score2': 0, 'score1i': 2, 'score2i': 0, 'goals1': [{'name': 'Gazinsky', 'minute': 12, 'score1': 1, 'score2': 0}, {'name': 'Cheryshev', 'minute': 43, 'score1': 2, 'score2': 0}, {'name': 'Dzyuba', 'minute': 71, 'score1': 3, 'score2': 0}, {'name': 'Cheryshev', 'minute': 90, 'offset': 1, 'score1': 4, 'score2': 0}, {'name': 'Golovin', 'minute': 90, 'offset': 4, 'score1': 5, 'score2': 0}], 'goals2': [], 'group': 'Group A', 'stadium': {'key': 'luzhniki', 'name': 'Luzhniki Stadium'}, 'city': 'Moscow', 'timezone': 'UTC+3'}]}


Organization of the world cup data, if you have done everything correctly, should be:
- world_cup_data is a dictionary with two keys, 'name' and 'rounds'.
- The 'name' key contains the name of the dataset.
- The 'round' key contains all match info organized by match day.


### Extracting Rounds

The value associated with the 'rounds' key is a list containing all of the actual information about the rounds and the matches within those rounds.

In [37]:
# CodeGrade step0
# Run this cell without changes
rounds = world_cup_data["rounds"]

rounds[3]

{'name': 'Matchday 4',
 'matches': [{'num': 9,
   'date': '2018-06-17',
   'time': '21:00',
   'team1': {'name': 'Brazil', 'code': 'BRA'},
   'team2': {'name': 'Switzerland', 'code': 'SUI'},
   'score1': 1,
   'score2': 1,
   'score1i': 1,
   'score2i': 0,
   'goals1': [{'name': 'Coutinho', 'minute': 20, 'score1': 1, 'score2': 0}],
   'goals2': [{'name': 'Zuber', 'minute': 50, 'score1': 1, 'score2': 1}],
   'group': 'Group E',
   'stadium': {'key': 'rostov', 'name': 'Rostov Arena'},
   'city': 'Rostov-on-Don',
   'timezone': 'UTC+3'},
  {'num': 10,
   'date': '2018-06-17',
   'time': '16:00',
   'team1': {'name': 'Costa Rica', 'code': 'CRC'},
   'team2': {'name': 'Serbia', 'code': 'SRB'},
   'score1': 0,
   'score2': 1,
   'score1i': 0,
   'score2i': 0,
   'goals1': [],
   'goals2': [{'name': 'Kolarov', 'minute': 56, 'score1': 0, 'score2': 1}],
   'group': 'Group E',
   'stadium': {'key': 'samara', 'name': 'Samara Arena'},
   'city': 'Samara',
   'timezone': 'UTC+4'},
  {'num': 11,
   

Translating this output into English:

Starting with the original `world_cup_data` dictionary, we used the key `"rounds"` to extract a list of rounds, which we assigned to the variable `rounds`.

`rounds` is a list of dictionaries. Each dictionary inside of `rounds` contains a name (e.g. `"Matchday 4"`) as well as a list of matches.

## Step 3: Extracting Matches

Now we can go one level deeper and extract all of the matches in the tournament. Because the round is irrelevant for this analysis, we can loop over all rounds and combine all of their matches into a single list called matches.

**Hint:** This is a good use case for using the `.extend` list method rather than `.append`, since we want to combine several lists of dictionaries into a single list of dictionaries, not a list of lists of dictionaries. [Documentation here.](https://docs.python.org/3/tutorial/datastructures.html#more-on-lists)

In [38]:
# CodeGrade step3
# Replace None with your code
matches = []

# "round" is a built-in function in Python so we use "round_" instead
for round_ in rounds:
    # Extract the list of matches for this round
    round_matches = round_['matches']
    # Add them to the overall list of matches
    matches.extend(round_matches)

matches[0]

{'num': 1,
 'date': '2018-06-14',
 'time': '18:00',
 'team1': {'name': 'Russia', 'code': 'RUS'},
 'team2': {'name': 'Saudi Arabia', 'code': 'KSA'},
 'score1': 5,
 'score2': 0,
 'score1i': 2,
 'score2i': 0,
 'goals1': [{'name': 'Gazinsky', 'minute': 12, 'score1': 1, 'score2': 0},
  {'name': 'Cheryshev', 'minute': 43, 'score1': 2, 'score2': 0},
  {'name': 'Dzyuba', 'minute': 71, 'score1': 3, 'score2': 0},
  {'name': 'Cheryshev', 'minute': 90, 'offset': 1, 'score1': 4, 'score2': 0},
  {'name': 'Golovin', 'minute': 90, 'offset': 4, 'score1': 5, 'score2': 0}],
 'goals2': [],
 'group': 'Group A',
 'stadium': {'key': 'luzhniki', 'name': 'Luzhniki Stadium'},
 'city': 'Moscow',
 'timezone': 'UTC+3'}

In [39]:
# Run this cell without changes

# There should be 64 matches. If the length is 20, that means
# you have a list of lists instead of a list of dictionaries
assert len(matches) == 64

# Each match in the list should be a dictionary
assert type(matches[0]) == dict

## Step 4: Extracting Teams

Each match has a `team1` and a `team2` - each with an associated name.

In [40]:
# Run this cell without changes
print(matches[0]["team1"])
print(matches[0]["team2"])

{'name': 'Russia', 'code': 'RUS'}
{'name': 'Saudi Arabia', 'code': 'KSA'}


Create a list of all unique team names by looping over every match in `matches` and adding the `"name"` values associated with both `team1` and `team2`. First creatr a Python set() containing the names of all unique teams playing in the World Cup. This set should be named team_set. This set will be used to create an alphabetically ordered list teams.


We'll use a `set` data type ([documentation here](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset)) to ensure unique teams, then convert it to a sorted list at the end.

In [41]:
# CodeGrade step4
# Replace None with appropriate code
teams_set = set()

for match in matches:
    # Add team1 name value to teams_set
    teams_set.add(match['team1']['name'])
    # Add team2 name value to teams_set
    teams_set.add(match['team2']['name'])

# Convert set to list and sort alphabetically
teams = sorted(list(teams_set))

In [42]:
# Run this cell without changes

# teams should be a list, not a set
assert type(teams) == list

# 32 teams competed in the 2018 World Cup
assert len(teams) == 32

# Each element of teams should be a string
# (the name), not a dictionary
assert type(teams[0]) == str

## Step 5: Create Data Structure to Associate Country with Performance

We now have unique identifiers (names) for each of our countries. These will be the keys in our dictionary structure that we want to create.

Start by initializing a dictionary called combined_data containing:
- keys: the strings from teams
- values: a dictionary containing the key 'wins' with the associated value 0

Initially `combined_data` will look something like this:
```
{
  'Argentina': { 'wins': 0 },
  ...
  'Uruguay':   { 'wins': 0 }
}
```

In [43]:
# CodeGrade step5
# Replace None with appropriate code

# Create the variable combined_data as described above
combined_data = {team: {"wins" : 0} for team in teams}

In [44]:
# Run this cell without changes

# combined_data should be a dictionary
assert type(combined_data) == dict

# the keys should be strings
assert type(list(combined_data.keys())[0]) == str

# the values should be dictionaries
assert combined_data["Japan"] == {"wins": 0}

## Step 6: Calculating Winner

Now it's time to revisit the `matches` list from earlier, in order to associate a team with the number of times it has won a match.

This time, let's write some functions to help organize our logic.

Write a function `find_winner` that takes in a `match` dictionary, and returns the name of the team that won the match.  Recall that a match is structured like this:

```
{
  'num': 1,
  'date': '2018-06-14',
  'time': '18:00',
  'team1': { 'name': 'Russia',       'code': 'RUS' },
  'team2': { 'name': 'Saudi Arabia', 'code': 'KSA' },
  'score1': 5,
  'score2': 0,
  'score1i': 2,
  'score2i': 0,
  'goals1': [
    { 'name': 'Gazinsky',  'minute': 12, 'score1': 1, 'score2': 0 },
    { 'name': 'Cheryshev', 'minute': 43, 'score1': 2, 'score2': 0 },
    { 'name': 'Dzyuba',    'minute': 71, 'score1': 3, 'score2': 0 },
    { 'name': 'Cheryshev', 'minute': 90, 'offset': 1, 'score1': 4, 'score2': 0 },
    { 'name': 'Golovin',   'minute': 90, 'offset': 4, 'score1': 5, 'score2': 0 }
  ],
  'goals2': [],
  'group': 'Group A',
  'stadium': { 'key': 'luzhniki', 'name': 'Luzhniki Stadium' },
  'city': 'Moscow',
  'timezone': 'UTC+3'
}
```

The winner is determined by comparing the values associated with the `'score1'` and `'score2'` keys. If score 1 is larger, then the name associated with the `'team1'` key is the winner. If score 2 is larger, then the name associated with the `'team2'` key is the winner. If the values are the same, there is no winner, so return `None`. (Unlike the group round of the World Cup, we are only counting *wins* as our "performance" construct, not 3 points for a win and 1 point for a tie.)

In [45]:
# CodeGrade step6
# Replace None with appropriate code

def find_winner(match):
    """
    Given a dictionary containing information about a match,
    return the name of the winner (or None in the case of a tie)
    """
    if match['score1'] > match['score2']:
        return match['team1']['name']
    elif match['score2'] > match['score1']:
        return match['team2']['name']
    else:
        return None

In [46]:
# Run this cell without changes
assert find_winner(matches[0]) == "Russia"
assert find_winner(matches[1]) == "Uruguay"
assert find_winner(matches[2]) == None

## Step 7: Counting and Adding Wins to Dictionary

Now that we have this helper function:
- Loop over every match in `matches`
- Find the winner
- Add 1 to the associated count of wins in `combined_data`.
- If the winner is `None`, skip adding it to the dictionary.

In [47]:
# CodeGrade step7
# Replace None with appropriate code

for match in matches:
    # Get the name of the winner
    winner = find_winner(match)
    # Only proceed to the next step if there was a winner
    if winner:
        # Add 1 to the associated count of wins
        combined_data[winner]['wins'] += 1

# Visually inspect the output to ensure the wins are different for different countries
combined_data

{'Argentina': {'wins': 1},
 'Australia': {'wins': 0},
 'Belgium': {'wins': 6},
 'Brazil': {'wins': 3},
 'Colombia': {'wins': 2},
 'Costa Rica': {'wins': 0},
 'Croatia': {'wins': 3},
 'Denmark': {'wins': 1},
 'Egypt': {'wins': 0},
 'England': {'wins': 3},
 'France': {'wins': 6},
 'Germany': {'wins': 1},
 'Iceland': {'wins': 0},
 'Iran': {'wins': 1},
 'Japan': {'wins': 1},
 'Mexico': {'wins': 2},
 'Morocco': {'wins': 0},
 'Nigeria': {'wins': 1},
 'Panama': {'wins': 0},
 'Peru': {'wins': 1},
 'Poland': {'wins': 1},
 'Portugal': {'wins': 1},
 'Russia': {'wins': 2},
 'Saudi Arabia': {'wins': 1},
 'Senegal': {'wins': 1},
 'Serbia': {'wins': 1},
 'South Korea': {'wins': 1},
 'Spain': {'wins': 1},
 'Sweden': {'wins': 3},
 'Switzerland': {'wins': 1},
 'Tunisia': {'wins': 1},
 'Uruguay': {'wins': 4}}

## Associating Countries with 2018 Population

Next, you will need to add the 2018 population for each country to the existing data structure so that it also connects each country name to its 2018 population.

Recall that `combined_data` currently looks something like this:
```
{
  'Argentina': { 'wins': 1 },
  ...
  'Uruguay':   { 'wins': 4 }
}
```

And the goal is for it to look something like this:
```
{
  'Argentina': { 'wins': 1, 'population': 44494502 },
  ...
  'Uruguay':   { 'wins': 4, 'population': 3449299  }
}
```

To do that, we need to extract the 2018 population information from the CSV data.

### Exploring the Structure of the Population Data CSV

Recall that previously we loaded information from a CSV containing population data into a list of dictionaries called `population_data`.

In [48]:
# Run this cell without changes
len(population_data)

12695

12,695 is a very large number of rows to print out, so let's look at some samples instead.

In [49]:
# Run this cell without changes
np.random.seed(42)
population_record_samples = np.random.choice(population_data, size=10)
population_record_samples

array([{'': '9984', 'Country Name': 'Malta', 'Country Code': 'MLT', 'Year': '1983', 'Value': '330524'},
       {'': '3574', 'Country Name': 'Bahrain', 'Country Code': 'BHR', 'Year': '1994', 'Value': '549583'},
       {'': '8104', 'Country Name': 'Iran, Islamic Rep.', 'Country Code': 'IRN', 'Year': '1988', 'Value': '53077313'},
       {'': '7905', 'Country Name': 'Iceland', 'Country Code': 'ISL', 'Year': '1966', 'Value': '195570'},
       {'': '14678', 'Country Name': 'United Arab Emirates', 'Country Code': 'ARE', 'Year': '1966', 'Value': '159976'},
       {'': '13998', 'Country Name': 'Thailand', 'Country Code': 'THA', 'Year': '1994', 'Value': '58875269'},
       {'': '8448', 'Country Name': 'Jamaica', 'Country Code': 'JAM', 'Year': '1978', 'Value': '2105907'},
       {'': '8979', 'Country Name': 'Kuwait', 'Country Code': 'KWT', 'Year': '1978', 'Value': '1224067'},
       {'': '3180', 'Country Name': 'Argentina', 'Country Code': 'ARG', 'Year': '2013', 'Value': '42202935'},
       {'': 

There are **2 filtering tasks**, **1 data normalization task**, and **1 type conversion task** to be completed, based on what we can see in this sample. We'll walk through each of them below.

(In a more realistic data cleaning environment, you most likely won't happen to get a sample that demonstrates all of the data cleaning steps needed, but this sample was chosen carefully for example purposes.)

## Step 8: Filtering Population Data

We already should have suspected that this dataset would require some filtering, since there are 32 records in our current `combined_data` dataset and 12,695 records in `population_data`. Now that we have looked at this sample, we can identify 2 features we'll want to use in order to filter down the `population_data` records to just 32.

The two features to filter on are ***`'Country Name'`*** *and* ***`'Year'`***. We can see from the sample above that there are countries in `population_data` that are not present in `combined_data` (e.g. Malta) and there are years present that are not 2018.

Create a new variable `population_data_filtered` that only includes relevant records from `population_data`. Relevant records are records where the country name is one of the countries in the `teams` list, and the year is "2018".

(It's okay to leave 2018 as a string since we are not performing any math operations on it, just make sure you check for `"2018"` and not `2018`.)

In [50]:
# CodeGrade step8
# Replace None with appropriate code

population_data_filtered = []

for record in population_data:
    # Add record to population_data_filtered if relevant
    if record['Country Name'] in teams and record['Year'] == '2018':
        population_data_filtered.append(record)
    
len(population_data_filtered)  # 27

27

Hmm...what went wrong? Why do we only have 27 records, and not 32?

Did we really get a dataset with 12k records that's missing 5 of the data points we need?

Let's take a closer look at the population data samples again, specifically the third one:

In [51]:
# Run this cell without changes
population_record_samples[2]

{'': '8104',
 'Country Name': 'Iran, Islamic Rep.',
 'Country Code': 'IRN',
 'Year': '1988',
 'Value': '53077313'}

And compare that with the value for Iran in `teams`:

In [52]:
# Run this cell without changes
teams[13]

'Iran'

Ohhhh...we have a data normalization issue. One dataset refers to this country as `'Iran, Islamic Rep.'`, while the other refers to it as `'Iran'`. This is a common issue we face when using data about countries and regions, where there is no universally-accepted naming convention.

## Step 9: Normalizing Locations in Population Data

Sometimes data normalization can be a very, very time-consuming task where you need to find "crosswalk" data that can link the two formats together, or you need to write advanced regex formulas to line everything up.

For this task, there are only 5 missing, so we'll just go ahead and give you a function that makes the appropriate substitutions.

In [53]:
# CodeGrade step0
# Run this cell without changes
def normalize_location(country_name):
    """
    Given a country name, return the name that the
    country uses when playing in the FIFA World Cup
    """
    name_sub_dict = {
        "Russian Federation": "Russia",
        "Egypt, Arab Rep.": "Egypt",
        "Iran, Islamic Rep.": "Iran",
        "Korea, Rep.": "South Korea",
        "United Kingdom": "England",
    }
    # The .get method returns the corresponding value from
    # the dict if present, otherwise returns country_name
    return name_sub_dict.get(country_name, country_name)

In [54]:
# Run this cell without changes
# Example where normalized location is different
print(normalize_location("Russian Federation"))
# Example where normalized location is the same
print(normalize_location("Argentina"))

Russia
Argentina


Now, write new code to create `population_data_filtered` with normalized country names.

In [62]:
# CodeGrade step9
# Replace None with appropriate code

population_data_filtered = []

for record in population_data:
    # Get normalized country name
    country = normalize_location(record['Country Name'])
    # Add record to population_data_filtered if relevant
    if country in teams and record['Year'] == '2018':
        # Replace the country name in the record
        record['Country Name'] = country
        # Append to list
        population_data_filtered.append(record)

len(population_data_filtered)  # 32

32

Great, now we should have 32 records instead of 27.

## Step 10: Type Conversion of Population Data

We need to do one more thing before we'll have population data that is usable for analysis. Take a look at this record from `population_data_filtered` to see if you can spot it:

In [56]:
# Run this cell without changes
population_data_filtered[0]

{'': '3185',
 'Country Name': 'Argentina',
 'Country Code': 'ARG',
 'Year': '2018',
 'Value': '44494502'}

Every key has the same data type (`str`), including the population value. In this example, it's `'44494502'`, when it needs to be `44494502` if we want to be able to compute statistics with it.

Loop over `population_data_filtered` and convert the data type of the value associated with the `"Value"` key from a string to an integer.

In [57]:
# CodeGrade step10
# Replace None with appropriate code
for record in population_data_filtered:
    # Convert the population value from str to int
    record['Value'] = int(record['Value'])

# Look at the last record to make sure the population value is an int
population_data_filtered[-1]

{'': '14907',
 'Country Name': 'Uruguay',
 'Country Code': 'URY',
 'Year': '2018',
 'Value': 3449299}

In [58]:
# Run this cell without changes
assert type(population_data_filtered[-1]["Value"]) == int

## Step 11: Adding Population Data

Now it's time to add the population data to `combined_data`. Recall that the data structure currently looks like this:

In [59]:
# Run this cell without changes
combined_data

{'Argentina': {'wins': 1},
 'Australia': {'wins': 0},
 'Belgium': {'wins': 6},
 'Brazil': {'wins': 3},
 'Colombia': {'wins': 2},
 'Costa Rica': {'wins': 0},
 'Croatia': {'wins': 3},
 'Denmark': {'wins': 1},
 'Egypt': {'wins': 0},
 'England': {'wins': 3},
 'France': {'wins': 6},
 'Germany': {'wins': 1},
 'Iceland': {'wins': 0},
 'Iran': {'wins': 1},
 'Japan': {'wins': 1},
 'Mexico': {'wins': 2},
 'Morocco': {'wins': 0},
 'Nigeria': {'wins': 1},
 'Panama': {'wins': 0},
 'Peru': {'wins': 1},
 'Poland': {'wins': 1},
 'Portugal': {'wins': 1},
 'Russia': {'wins': 2},
 'Saudi Arabia': {'wins': 1},
 'Senegal': {'wins': 1},
 'Serbia': {'wins': 1},
 'South Korea': {'wins': 1},
 'Spain': {'wins': 1},
 'Sweden': {'wins': 3},
 'Switzerland': {'wins': 1},
 'Tunisia': {'wins': 1},
 'Uruguay': {'wins': 4}}

The goal is for it to be structured like this:
```
{
  'Argentina': { 'wins': 1, 'population': 44494502 },
  ...
  'Uruguay':   { 'wins': 4, 'population': 3449299  }
}
```

In the cell below, loop over `population_data_filtered` and add information about population to each country in `combined_data`:

In [60]:
# CodeGrade step11
# Replace None with appropriate code
for record in population_data_filtered:
    # Extract the country name from the record
    country = record['Country Name']
    # Extract the population value from the record
    population = record['Value']
    # Add this information to combined_data
    combined_data[country]['population'] = population

# Look combined_data
combined_data

{'Argentina': {'wins': 1, 'population': 44494502},
 'Australia': {'wins': 0, 'population': 24982688},
 'Belgium': {'wins': 6, 'population': 11433256},
 'Brazil': {'wins': 3, 'population': 209469333},
 'Colombia': {'wins': 2, 'population': 49648685},
 'Costa Rica': {'wins': 0, 'population': 4999441},
 'Croatia': {'wins': 3, 'population': 4087843},
 'Denmark': {'wins': 1, 'population': 5793636},
 'Egypt': {'wins': 0, 'population': 98423595},
 'England': {'wins': 3, 'population': 66460344},
 'France': {'wins': 6, 'population': 66977107},
 'Germany': {'wins': 1, 'population': 82905782},
 'Iceland': {'wins': 0, 'population': 352721},
 'Iran': {'wins': 1, 'population': 81800269},
 'Japan': {'wins': 1, 'population': 126529100},
 'Mexico': {'wins': 2, 'population': 126190788},
 'Morocco': {'wins': 0, 'population': 36029138},
 'Nigeria': {'wins': 1, 'population': 195874740},
 'Panama': {'wins': 0, 'population': 4176873},
 'Peru': {'wins': 1, 'population': 31989256},
 'Poland': {'wins': 1, 'popu

In [61]:
# Run this cell without changes
assert type(combined_data["Uruguay"]) == dict
assert type(combined_data["Uruguay"]["population"]) == int

## Summary

Congratulations! That was a long lab, pulling together a lot of material. You read data into Python, extracted the relevant information, cleaned the data, and combined the data into a new format to be used in analysis. While we will continue to introduce new tools and techniques, these essential steps will be present for the rest of your data science projects from here on out!